### Prerequisites

1. Have an OpenAI api key
2. Have an Ngrok token
3. Ensure that skincare_knowledge_base.zip has been uploaded to the colab environment

In [ ]:
!pip install llama-index langchain openai tqdm typing -q
!pip install flask flask-ngrok -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


### Imports

In [ ]:
import os
import zipfile

import torch

from llama_index.llms.openai.base import OpenAI

from llama_index.core import SimpleDirectoryReader, Settings
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.core.vector_stores.simple import SimpleVectorStore
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.base.embeddings.base import BaseEmbedding

from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

from transformers import AutoModel, AutoTokenizer
from typing import List
import torch

### Project keys and configurations

In [ ]:
API_KEY = "your_open_ai_api_key"
os.environ["OPENAI_API_KEY"] = "your_open_ai_api_key"
Settings.llm = OpenAI(model='gpt-4o')


### Initialize knowledge base

#### Extract the zip file and get all skin care recommendation data

In [ ]:
def extract_zip(zip_file, destination_folder):
    if not os.path.exists(zip_file):
        print(f"ZIP file '{zip_file}' does not exist.")
        return

    os.makedirs(destination_folder, exist_ok=True)

    try:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(destination_folder)
        print(f"Extracted '{zip_file}' to '{destination_folder}'.")
    except zipfile.BadZipFile:
        print(f"Error: '{zip_file}' is not a valid ZIP file.")
    except Exception as e:
        print(f"Error extracting '{zip_file}': {e}")

if __name__ == "__main__":
    zip_file = "skincare_knowledge_base.zip"
    destination_folder = "skincare_knowledge_base"
    extract_zip(zip_file, destination_folder)


Extracted 'skincare_knowledge_base.zip' to 'skincare_knowledge_base'.


To use GPU, we need to implement a concrete child class of BaseEmbedding in LlamaIndex. This acts as a bridge between AutoModel/Tokenizer and Llama framework

In [ ]:
from transformers import AutoModel, AutoTokenizer
from typing import List
import torch

class CustomEmbedding(BaseEmbedding):
    def __init__(self, model_name: str = "sentence-transformers/multi-qa-mpnet-base-dot-v1", **kwargs):
        """
        Initialize the CustomEmbedding class with a 1536-dimensional model.

        Args:
            model_name (str): Hugging Face model name that outputs 1536-dimensional embeddings.
        """
        super().__init__(**kwargs)
        self._tokenizer = AutoTokenizer.from_pretrained(model_name)
        self._model = AutoModel.from_pretrained(model_name).to("cuda")

    def _get_query_embedding(self, query: str) -> List[float]:
        """
        Generate embeddings for a single query.

        Args:
            query (str): Input query string.

        Returns:
            List[float]: 1536-dimensional embedding for the query.
        """
        return self._get_text_embedding(query)

    def _get_text_embedding(self, text: str) -> List[float]:
        """
        Generate embeddings for a single text.

        Args:
            text (str): Input text string.

        Returns:
            List[float]: 1536-dimensional embedding for the text.
        """
        inputs = self._tokenizer([text], padding=True, truncation=True, return_tensors="pt").to("cuda")
        with torch.no_grad():
            embeddings = self._model(**inputs).last_hidden_state.mean(dim=1).cpu().numpy()
        return embeddings[0].tolist()

    def _get_text_embeddings(self, texts: List[str]) -> List[List[float]]:
        """
        Generate embeddings for a batch of texts.

        Args:
            texts (List[str]): List of input text strings.

        Returns:
            List[List[float]]: Batch of 1536-dimensional embeddings.
        """
        inputs = self._tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to("cuda")
        with torch.no_grad():
            embeddings = self._model(**inputs).last_hidden_state.mean(dim=1).cpu().numpy()
        return embeddings.tolist()

    async def _aget_query_embedding(self, query: str) -> List[float]:
        """
        Asynchronous implementation of generating embeddings for a single query.

        Args:
            query (str): Input query string.

        Returns:
            List[float]: 1536-dimensional embedding for the query.
        """
        return self._get_query_embedding(query)

    async def _aget_text_embedding(self, text: str) -> List[float]:
        """
        Asynchronous implementation of generating embeddings for a single text.

        Args:
            text (str): Input text string.

        Returns:
            List[float]: 1536-dimensional embedding for the text.
        """
        return self._get_text_embedding(text)

    async def _aget_text_embeddings(self, texts: List[str]) -> List[List[float]]:
        """
        Asynchronous implementation of generating embeddings for a batch of texts.

        Args:
            texts (List[str]): List of input text strings.

        Returns:
            List[List[float]]: Batch of 1536-dimensional embeddings.
        """
        return self._get_text_embeddings(texts)


1. Load the documents, utilize the CustomEmbedding class to vector knowledge store, and save it as an object of VectorStoreIndex.

2. This would then be reloaded to provide necessary rag content upon having a user query

In [ ]:
documents = SimpleDirectoryReader("./skincare_knowledge_base/").load_data()

In [ ]:
embedding_model = CustomEmbedding(model_name="sentence-transformers/multi-qa-mpnet-base-dot-v1")
index = VectorStoreIndex.from_documents(documents, embed_model=embedding_model)

Set the name of VectorIndex and save index to disk


In [ ]:
index.set_index_id("vector_index")
index.storage_context.persist(persist_dir="./storage")

#### Load the index to use the skin care knowledge base

In [ ]:
storage_context = StorageContext.from_defaults(persist_dir="storage")
index = load_index_from_storage(storage_context, index_id="vector_index")


### Retrieving top k most relevant documents

In [ ]:
query_engine = index.as_query_engine(
    embed_model=embedding_model,
    similarity_top_k=10,
)


### Skincare assistant prompt template

In [ ]:
standard_skincare_template = """Factor in different types of products used in skincare.
 We need all products for a dedicated skin routine. Explain what each skin care product does and why do we need it,
 and generate a skincare routine as well. A skincare routine should consist of 2 phases - a morning skincare routine and a nighttime routine.
 Analyze what products should be applied when - for eg: retinol is only applied at nighttime, to minimize sun damage whereas vitamin C is applied in the morning.
 If there's a recommendation that requires prescription, or triggers allergies, alert the user regarding the same.
 """

### Sanity testing

In [ ]:
test_case_1 = """Recommend me some skin care prducts for dry, sensitive skin. """

In [ ]:
response = query_engine.query(standard_skincare_template + test_case_1)
print(response)

For a comprehensive skincare routine, various products play essential roles in maintaining healthy skin. Here is a breakdown of different skincare products and their purposes:

1. Cleanser: Cleansers remove dirt, oil, and impurities from the skin, preparing it for other products.
2. Toner: Toners help balance the skin's pH levels and can provide additional hydration.
3. Serum: Serums are concentrated formulas that target specific skin concerns like hydration, brightening, or anti-aging.
4. Moisturizer: Moisturizers hydrate the skin, lock in moisture, and create a protective barrier.
5. Sunscreen: Sunscreen protects the skin from harmful UV rays, preventing premature aging and skin damage.
6. Retinol: Retinol is a powerful ingredient that promotes skin renewal and reduces the appearance of fine lines and wrinkles.
7. Vitamin C: Vitamin C brightens the skin, evens out skin tone, and provides antioxidant protection.

For a skincare routine tailored to dry, sensitive skin, here is a sugges

In [ ]:
response = query_engine.query(standard_skincare_template + test_case_1)
print(response)

For a comprehensive skincare routine, various products play essential roles in maintaining healthy skin. Here is a breakdown of different skincare products and their purposes:

1. Cleanser: Cleansers remove dirt, oil, and impurities from the skin, preparing it for other products.
2. Toner: Toners help balance the skin's pH levels and can provide additional hydration.
3. Serum: Serums are concentrated formulas that target specific skin concerns like hydration, brightening, or anti-aging.
4. Moisturizer: Moisturizers hydrate the skin, lock in moisture, and create a protective barrier.
5. Sunscreen: Sunscreen protects the skin from harmful UV rays, preventing premature aging and skin damage.
6. Exfoliator: Exfoliators remove dead skin cells, promoting cell turnover and revealing smoother skin.
7. Treatment Products: Targeted treatments like retinol or acne spot treatments address specific skin concerns.

For a skincare routine tailored to dry, sensitive skin, here is a suggested regimen:


### Util script to update path

Since llamaIndex is a library that changes its file locations frequently, code to do a dfs search over the python module folder

In [ ]:
import os
import re
import importlib.util

def find_class_in_package(package_name, class_name):
    try:
        spec = importlib.util.find_spec(package_name)
        if not spec or not spec.submodule_search_locations:
            print(f"Package '{package_name}' not found.")
            return None

        package_path = next(iter(spec.submodule_search_locations))
        class_imports = []

        for root, _, files in os.walk(package_path):
            for file in files:
                if file.endswith(".py"):
                    file_path = os.path.join(root, file)
                    with open(file_path, 'r', encoding='utf-8') as f:
                        content = f.read()

                    if f'class {class_name}' in content:
                        module_path = os.path.relpath(file_path, package_path).replace(os.sep, '.').removesuffix('.py')
                        import_statement = f"from {package_name}.{module_path} import {class_name}"
                        class_imports.append(import_statement)

        if class_imports:
            return class_imports
        else:
            print(f"Class '{class_name}' not found in package '{package_name}'.")
            return None

    except Exception as e:
        print(f"Error occurred: {e}")
        return None


if __name__ == "__main__":
    package = "llama_index"
    class_to_find = "OpenAI"

    result = find_class_in_package(package, class_to_find)
    if result:
        print("Found import statements:")
        for import_statement in result:
            print(import_statement)


Found import statements:
from llama_index.legacy.finetuning.openai.base import OpenAI
from llama_index.legacy.agent.openai_assistant_agent import OpenAI
from llama_index.legacy.agent.legacy.openai_agent import OpenAI
from llama_index.legacy.agent.openai.base import OpenAI
from llama_index.legacy.agent.openai.step import OpenAI
from llama_index.legacy.llms.openai_like import OpenAI
from llama_index.legacy.llms.openai import OpenAI
from llama_index.legacy.multi_modal_llms.openai import OpenAI
from llama_index.legacy.callbacks.finetuning_handler import OpenAI
from llama_index.legacy.embeddings.openai import OpenAI
from llama_index.legacy.question_gen.openai_generator import OpenAI
from llama_index.legacy.program.openai_program import OpenAI
from llama_index.agent.openai.openai_assistant_agent import OpenAI
from llama_index.agent.openai.base import OpenAI
from llama_index.agent.openai.step import OpenAI
from llama_index.llms.openai.base import OpenAI
from llama_index.multi_modal_llms.opena

### Installing ngrok on colab

In [ ]:
!wget -q -O ngrok.zip https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip
!unzip -o ngrok.zip
!chmod +x ngrok

Archive:  ngrok.zip
  inflating: ngrok                   


In [ ]:
!./ngrok config add-authtoken [your_ngrok_token_here]

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


### Flask web app: To expose skincare rag llm as a web application

Note: To run the web app correctly, you must:
1. Create a templates folder and upload index.html.
2. Create a static folder and upload typing_bubble.gif

Both of these folders are provided in the github repository.


In [ ]:
import threading
from flask import Flask, render_template, request
import os
import logging
import requests

app = Flask(__name__)

log_file = "app.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(log_file, mode="a"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

stop_flag = threading.Event()

def get_openai_response(user_input):
    try:
        logger.info(f"Received input: {user_input}")
        response = query_engine.query(standard_skincare_template + user_input)
        logger.info(f"Generated response: {response}")
        return response
    except Exception as e:
        logger.error(f"Error while processing input: {str(e)}")
        return f"Error: {str(e)}"

def process_response(response):
    general_advice = "No general advice provided."
    morning_routine = "No morning routine provided."
    night_routine = "No night routine provided."

    lower_response = response.lower()
    morning_index = lower_response.find("morning")
    night_index = lower_response.find("night")

    if morning_index != -1:
        general_advice = response[:morning_index].strip()
        if night_index != -1:
            morning_routine = response[morning_index:night_index].strip()
            night_routine = response[night_index:].strip()
        else:
            morning_routine = response[morning_index:].strip()
    elif night_index != -1:
        general_advice = response[:night_index].strip()
        night_routine = response[night_index:].strip()
    else:
        general_advice = response.strip()

    logger.info("Response split into sections.")
    logger.info(f"General Advice: {general_advice}")
    logger.info(f"Morning Routine: {morning_routine}")
    logger.info(f"Night Routine: {night_routine}")

    return general_advice, morning_routine, night_routine

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        user_prompt = request.form["prompt"]
        logger.info("Handling POST request.")
        response = get_openai_response(user_prompt)
        general_advice, morning_routine, night_routine = process_response(response)
        return render_template(
            "index.html",
            prompt=user_prompt,
            general_advice=general_advice,
            morning_routine=morning_routine,
            night_routine=night_routine,
        )
    logger.info("Serving GET request.")
    return render_template("index.html", prompt="", general_advice="", morning_routine="", night_routine="")

@app.route('/hello')
def hello():
    logger.info("Accessed '/hello' route.")
    return "Hello, Flask is running in the background with ngrok!"

@app.route("/shutdown", methods=["POST"])
def shutdown():
    func = request.environ.get('werkzeug.server.shutdown')
    if func is None:
        raise RuntimeError("Not running the Werkzeug Server")
    logger.info("Shutting down Flask server.")
    stop_flag.set()
    func()
    return "Server shutting down..."

def run_app():
    logger.info("Starting ngrok.")
    os.system("./ngrok http 5000 &")
    logger.info("Starting Flask app.")
    app.run(host="0.0.0.0", port=5000)

flask_thread = threading.Thread(target=run_app, name="FlaskThread")
flask_thread.setDaemon(True)
flask_thread.start()

logger.info("Flask app is running in the background. Continue with other notebook cells.")

def stop_flask_thread():
    try:
        logger.info("Attempting to shut down Flask thread.")
        response = requests.post("http://127.0.0.1:5000/shutdown")
        logger.info(response.text)
    except Exception as e:
        logger.error(f"Error shutting down Flask thread: {e}")

Flask app is running in the background. Continue with other notebook cells.


<ipython-input-159-abe1920aaa70>:49: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  flask_thread.setDaemon(True)  # Set as a daemon thread


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


#### Dev testing only: Utils related to terminating flask web app sitting behind ngrok endpoint

In [ ]:
import os
import signal

# Get the current process's PID
current_pid = os.getpid()
current_pid

895

In [ ]:
import psutil
import os

def list_child_processes():
    current_process = psutil.Process(os.getpid())
    children = current_process.children(recursive=True)
    for child in children:
        print(f"PID: {child.pid}, Name: {child.name()}, CMDLINE: {child.cmdline()}")

list_child_processes()

PID: 58375, Name: ngrok, CMDLINE: []


In [ ]:
def terminate_child_process(pid):
    try:
        child = psutil.Process(pid)
        child.terminate()
        print(f"Terminated process with PID: {pid}")
    except Exception as e:
        print(f"Error: {e}")

terminate_child_process(<PID>)  # Replace <PID> with the correct PID


In [ ]:
# !ps -ax | grep 'python3'
# !kill -9 58375

     64 ?        Z      0:12 [python3] <defunct>
     65 ?        S      0:05 python3 /usr/local/bin/colab-fileshim.py
    114 ?        Sl     0:12 /usr/bin/python3 /usr/local/bin/jupyter-notebook --debug --transport="i
    895 ?        Ssl    2:15 /usr/bin/python3 -m colab_kernel_launcher -f /root/.local/share/jupyter
    926 ?        Sl     0:30 /usr/bin/python3 /usr/local/lib/python3.10/dist-packages/debugpy/adapte
  59805 ?        S      0:00 /bin/bash -c ps -ax | grep 'python3'
  59807 ?        S      0:00 grep python3


In [ ]:
import threading

for thread in threading.enumerate():
    print(f"Thread: {thread.name}, Alive: {thread.is_alive()}")


Thread: MainThread, Alive: True
Thread: Thread-2 (_thread_main), Alive: True
Thread: Thread-3, Alive: True
Thread: Thread-1, Alive: True
Thread: _colab_inspector_thread, Alive: True
Thread: Thread-10, Alive: True
Thread: Thread-11 (run_app), Alive: True


In [ ]:
!ps aux | grep ngrok

root       57779  0.0  0.0   7376  3512 ?        S    07:16   0:00 /bin/bash -c ps aux | grep ngrok
root       57783  0.0  0.0   6484  2316 ?        S    07:16   0:00 grep ngrok


In [ ]:
import subprocess

def start_ngrok():
    process = subprocess.Popen(["./ngrok", "http", "5000"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(f"ngrok started with PID: {process.pid}")
    return process

ngrok_process = start_ngrok()


ngrok started with PID: 58375


In [ ]:
!curl localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line","ID":"e139350b06e1dba52395203939e9f4fd","uri":"/api/tunnels/command_line","public_url":"https://f35a-34-87-41-212.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:5000","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [ ]:
import requests

response = requests.get("http://localhost:4040/api/tunnels")
data = response.json()
public_url = data['tunnels'][0]['public_url']
print(f"Public URL: {public_url}")


Public URL: https://f35a-34-87-41-212.ngrok-free.app
